# 1. Import packages

In [1]:
import re
import string
from typing import Final, List

from IPython.core.display import display_html, HTML

from textattack import Attack, AttackArgs, Attacker
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions.classification import UntargetedClassification
from textattack.loggers import CSVLogger
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attacked_text import AttackedText
from textattack.transformations import WordSwap

/home/sypark/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Load model and dataset

In [2]:
# Load from my_model.py and my_dataset.py
from my_model import get_model
from my_dataset import dataset

model = get_model("cuda:3")

# 3. Define `CustomWordSwapEmbedding` class

In [3]:
class CustomWordSwapEmbedding(WordSwap):
    def __init__(self) -> None:
        hexword_regex: Final[re.Pattern] = re.compile(r"^[0-9a-fA-F]{4}$")
        self.replacement_words: List[str] = []

        with open("./models/encryptd_vocab.txt") as vocab_file:
            for vocab in vocab_file:
                self.replacement_words.append(
                    vocab.rstrip()
                ) if hexword_regex.fullmatch(vocab.rstrip()) else None
                continue
            pass

        return super().__init__(string.hexdigits)

    def _get_transformations(
        self, current_text: AttackedText, indices_to_modify: List[int]
    ) -> List[AttackedText]:
        words = current_text.words
        transformed_texts: List[AttackedText] = []

        for i in indices_to_modify:
            # if i < 22:
            #     continue
            
            word_to_replace = words[i]
            replacement_words = self._get_replacement_words(word_to_replace)
            transformed_texts_idx: List[AttackedText] = []

            for r in replacement_words:
                if r == word_to_replace:
                    continue

                indices: List[int] = []
                new_words: List[str] = []

                indices.append(i)
                new_words.append(r)

                if i < len(words) - 1:
                    post_r = r[2:] + words[i + 1][2:]
                    indices.append(i + 1)
                    new_words.append(post_r)
                    pass

                transformed_texts_idx.append(
                    current_text.replace_words_at_indices(indices, new_words)
                )
                continue

            transformed_texts.extend(transformed_texts_idx)
            continue

        return transformed_texts

    def _get_replacement_words(self, word: str) -> List[str]:
        replacement_words = [w for w in self.replacement_words if w.startswith(word[:2])]

        if word in replacement_words:
            replacement_words.remove(word)
            pass
        
        return replacement_words

    pass

# 4. Construct `textattack.Attack` object

In [4]:
# Construct the four fundamental components of the attack
goal_function = UntargetedClassification(model)
constraints = [
    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.9),
]
transformation = CustomWordSwapEmbedding()
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'my_model.Classifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


# 5. Construct `textattack.Attacker` object

In [5]:
# Attack until 1000 successful attacks are reached
attack_args = AttackArgs(num_successful_examples=100, random_seed=42)
attacker = Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CustomWordSwapEmbedding
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  1%|          | 1/100 [00:02<03:50,  2.33s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:02<04:23,  2.66s/it]

[[68 (100%)]] --> [[52 (49%)]]

[[e57a]] [[7af9]] f9d8 d828 2874 74fb fb4c 4ca4 a4af af80 8018 1801 01f5 f5f1 f112 1200 0000 0001 0101 0108 080a 0aad ad90 9064 6433 334d 4d8d 8d2b 2b08 0835 35eb eb46 462a 2a9b 9b2a 2a90 90a9 a97e 7e96 968d 8d4c 4c82 826a 6a6b 6b5a 5a88 888b 8b70 708a 8ad4 d4dd dd16 1697 97dc dc6c 6c88 8843 43a3 a3d1 d1f9 f929 29d6 d6ee eeb7

[[e501]] [[01f9]] f9d8 d828 2874 74fb fb4c 4ca4 a4af af80 8018 1801 01f5 f5f1 f112 1200 0000 0001 0101 0108 080a 0aad ad90 9064 6433 334d 4d8d 8d2b 2b08 0835 35eb eb46 462a 2a9b 9b2a 2a90 90a9 a97e 7e96 968d 8d4c 4c82 826a 6a6b 6b5a 5a88 888b 8b70 708a 8ad4 d4dd dd16 1697 97dc dc6c 6c88 8843 43a3 a3d1 d1f9 f929 29d6 d6ee eeb7




...

--------------------------------------------- Result 98 ---------------------------------------------
[[102 (94%)]] --> [[45 (100%)]]

01bb bb99 [[99b8]] [[b8c0]] c0cc cc24 2485 858f 8f29 2950 5018 1801 01fd fdda da63 6300 0000 0017 1703 0303 0304 041f 1f7b 7b3b 3bc1 c1cb cb3f 3f9b 9b40 403e 3ece cee5 e544 4431 316d 6d9d 9d7f 7fc7 c752 52c8 c894 945d 5d8b 8b06 06d1 d1d4 d451 519d 9d66 664b 4b7a 7aee ee38 3864 649a 9ade de7b 7b11 1135 3576 76ce ce47 4795 95c8

01bb bb99 [[9977]] [[77c0]] c0cc cc24 2485 858f 8f29 2950 5018 1801 01fd fdda da63 6300 0000 0017 1703 0303 0304 041f 1f7b 7b3b 3bc1 c1cb cb3f 3f9b 9b40 403e 3ece cee5 e544 4431 316d 6d9d 9d7f 7fc7 c752 52c8 c894 945d 5d8b 8b06 06d1 d1d4 d451 519d 9d66 664b 4b7a 7aee ee38 3864 649a 9ade de7b 7b11 1135 3576 76ce ce47 4795 95c8




[Succeeded / Failed / Skipped / Total] 96 / 0 / 3 / 99:  96%|█████████▌| 96/100 [03:09<00:07,  1.97s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[8 (99%)]] --> [[118 (89%)]]

01bb bb59 597e 7e63 63a9 a926 26ae aee9 e997 9750 5018 1808 0800 004d 4d3f 3f00 0000 0017 1703 0303 0300 [[0013]] [[1334]] 3420 209c 9c2f 2f74 748f 8f1e 1ea5 a54c 4ccd cde9 e993 935a 5a83 83ab aba9 a909 09d2 d2e2

01bb bb59 597e 7e63 63a9 a926 26ae aee9 e997 9750 5018 1808 0800 004d 4d3f 3f00 0000 0017 1703 0303 0300 [[00fa]] [[fa34]] 3420 209c 9c2f 2f74 748f 8f1e 1ea5 a54c 4ccd cde9 e993 935a 5a83 83ab aba9 a909 09d2 d2e2




[Succeeded / Failed / Skipped / Total] 97 / 0 / 3 / 100:  97%|█████████▋| 97/100 [03:11<00:05,  1.97s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[19 (100%)]] --> [[73 (100%)]]

9d72 7220 202a 2a56 566c 6ceb eb08 087a 7a10 1080 8010 1000 0053 5332 32ea ea00 0000 0001 0101 0108 080a 0a88 88c1 c1e2 e263 6302 [[029c]] [[9ccb]] cbd7 d774 7483 8322 2257 57f6 f66a 6aba ba6e 6ec8 c8aa aacd cd11 1133 33c0 c0e0 e094 94b0 b0cc cc06 063b 3b0c 0c47 47d6 d66f 6f31 31b4 b4f7 f731 31c3 c321 21eb eb9a 9a90 90ba ba01

9d72 7220 202a 2a56 566c 6ceb eb08 087a 7a10 1080 8010 1000 0053 5332 32ea ea00 0000 0001 0101 0108 080a 0a88 88c1 c1e2 e263 6302 [[02ec]] [[eccb]] cbd7 d774 7483 8322 2257 57f6 f66a 6aba ba6e 6ec8 c8aa aacd cd11 1133 33c0 c0e0 e094 94b0 b0cc cc06 063b 3b0c 0c47 47d6 d66f 6f31 31b4 b4f7 f731 31c3 c321 21eb eb9a 9a90 90ba ba01



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 97     |
| Number of failed attacks:     

# 6. Display attack results

In [6]:
# Display the attack results and the differences
logger = CSVLogger(color_method="html")
for result in attack_results:
    logger.log_attack_result(result)
    continue

display_html(HTML(logger.df[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,e57a 7af9 f9d8 d828 2874 74fb fb4c 4ca4 a4af af80 8018 1801 01f5 f5f1 f112 1200 0000 0001 0101 0108 080a 0aad ad90 9064 6433 334d 4d8d 8d2b 2b08 0835 35eb eb46 462a 2a9b 9b2a 2a90 90a9 a97e 7e96 968d 8d4c 4c82 826a 6a6b 6b5a 5a88 888b 8b70 708a 8ad4 d4dd dd16 1697 97dc dc6c 6c88 8843 43a3 a3d1 d1f9 f929 29d6 d6ee eeb7,e501 01f9 f9d8 d828 2874 74fb fb4c 4ca4 a4af af80 8018 1801 01f5 f5f1 f112 1200 0000 0001 0101 0108 080a 0aad ad90 9064 6433 334d 4d8d 8d2b 2b08 0835 35eb eb46 462a 2a9b 9b2a 2a90 90a9 a97e 7e96 968d 8d4c 4c82 826a 6a6b 6b5a 5a88 888b 8b70 708a 8ad4 d4dd dd16 1697 97dc dc6c 6c88 8843 43a3 a3d1 d1f9 f929 29d6 d6ee eeb7
1,01bb bb1a 1ae9 e93d 3dd1 d124 24ed edd8 d820 2080 8018 1808 0800 00cc cc94 9400 0000 0001 0101 0108 080a 0a3b 3b2c 2cbe be15 150b 0bc0 c0b9 b9f2 f217 1703 0303 0300 001f 1f6e 6eab ab12 12f7 f761 617c 7ccb cb67 675b 5b4f 4f8d 8de4 e421 21d0 d034 3409 09b7 b7e5 e52a 2ab1 b1bb bbf7 f758 5899 9978 7891 91e2 e217 1718 1807,01bb bb1a 1ae9 e93d 3dd1 d124 24ed edd8 d820 2080 8018 1808 0800 00cc cc94 9400 0000 0001 0101 0108 080a 0a3b 3b66 66be be15 150b 0bc0 c0b9 b9f2 f217 1703 0303 0300 001f 1f6e 6eab ab12 12f7 f761 617c 7ccb cb67 675b 5b4f 4f8d 8de4 e421 21d0 d034 3409 09b7 b7e5 e52a 2ab1 b1bb bbf7 f758 5899 9978 7891 91e2 e217 1718 1807
2,f0df dfff ff92 92fa fa22 22b6 b64c 4c94 94ac ac80 8018 1801 0110 1041 414f 4f00 0000 0001 0101 0108 080a 0a05 0542 42c9 c99a 9a29 2920 20aa aae5 e517 1703 0303 0302 020d 0d8c 8c3a 3abf bf43 4369 690b 0ba3 a3ed eda5 a5d7 d739 390e 0ee8 e82d 2d36 3686 86d3 d3b8 b895 95d9 d94a 4a73 73de de6e 6ea6 a6f9 f9e3 e34d 4dca ca1a,f0df df0f 0f92 92fa fa22 22b6 b64c 4c94 94ac ac80 8018 1801 01f3 f341 414f 4f00 0000 0001 0101 0108 080a 0a05 0542 42c9 c99a 9a29 2920 20aa aae5 e517 1703 0303 0302 020d 0d8c 8c3a 3abf bf43 4369 690b 0ba3 a3ed eda5 a5d7 d739 390e 0ee8 e82d 2d36 3686 86d3 d3b8 b895 95d9 d94a 4a73 73de de6e 6ea6 a6f9 f9e3 e34d 4dca ca1a
3,01bb bbc8 c8ff ff31 3158 582f 2f63 630a 0aaa aa80 8010 1001 01f5 f596 9682 8200 0000 0001 0101 0108 080a 0acb cbb7 b786 8687 871b 1bcf cf2b 2bc9 c9a2 a213 1304 041e 1ee0 e04a 4af2 f23e 3e69 6961 612b 2b65 65e6 e63b 3b77 77f8 f855 557e 7e22 221d 1d62 6217 1770 70a3 a352 527a 7afd fd2e 2ea6 a65f 5fed edd0 d0f5 f5bd bd65,010b 0bc8 c8ff ff31 3158 582f 2f63 630a 0aaa aa80 8010 1001 01f5 f596 9682 8200 0000 0001 0101 0108 080a 0acb cbb7 b786 8687 871b 1bcf cf2b 2bc9 c9a2 a213 1304 041e 1ee0 e04a 4af2 f23e 3e69 6961 612b 2b65 65e6 e63b 3b77 77f8 f855 557e 7e22 221d 1d62 6217 1770 70a3 a352 527a 7afd fd2e 2ea6 a65f 5fed edd0 d0f5 f5bd bd65
4,01bb bbc9 c973 735f 5f72 72dd ddd1 d121 215b 5b50 5018 1800 00ff ff6c 6c15 1500 0000 00d5 d523 2329 2996 969c 9ce1 e10d 0d81 81a4 a404 049f 9faf af55 55c2 c250 50b4 b404 04c1 c109 09b8 b817 17b7 b712 1254 541c 1c23 23da da55 55b1 b109 09c2 c225 2510 10f2 f24e 4ecc cc8f 8f27 277b 7bf9 f9af af07 075f 5f33 33a3 a33b 3b34,01bb bb07 0773 735f 5f72 72dd ddd1 d121 215b 5b50 5018 1800 00ff ff6c 6c15 1500 0000 00d5 d523 2329 2996 969c 9ce1 e10d 0d81 81a4 a404 049f 9faf af55 55c2 c250 50b4 b404 04c1 c109 09b8 b817 17b7 b712 1254 541c 1c23 23da da55 55b1 b109 09c2 c225 2510 10f2 f24e 4ecc cc8f 8f27 277b 7bf9 f9af af07 075f 5f33 33a3 a33b 3b34
5,01bb bb5d 5dc3 c39c 9cd0 d0e8 e8f3 f388 88d5 d550 5018 1802 0203 0388 8834 3400 0000 0017 1703 0303 0301 0171 718a 8a95 95c3 c3a0 a0c3 c37c 7c2f 2f1d 1d28 28f7 f70b 0bf5 f5d1 d15e 5ee0 e04c 4c28 2834 34a2 a267 676c 6c75 751a 1a7b 7b81 81dd dda4 a41d 1dda da1a 1ad5 d5f7 f7a6 a6ca ca7b 7ba7 a742 42ed eda5 a544 44cd cdaa,01bb bb5d 5d7f 7f9c 9cd0 d0e8 e8f3 f388 88d5 d550 5018 1802 0203 0388 8834 3400 0000 0017 1703 0303 0301 0171 718a 8a95 95c3 c3a0 a0c3 c37c 7c2f 2f1d 1d28 28f7 f70b 0bf5 f5d1 d15e 5ee0 e04c 4c28 2834 34a2 a267 676c 6c75 751a 1a7b 7b81 81dd dda4 a41d 1dda da1a 1ad5 d5f7 f7a6 a6ca ca7b 7ba7 a742 42ed eda5 a544 44cd cdaa
6,c616 16af afd0 d081 81d0 d042 42f7 f70d 0de0 e050 5010 1000 00c1 c19f 9ff7 f700 0000 0080 809b 9be2 e2d7 d74d 4d64 6